In [73]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm
import os

In [74]:
df = pd.read_feather('imgl.df').set_index('index').query('ds_type == "train"')
func = lambda s: s[s.find("images/")+7:]
df["local_path"] = np.array([[f"../seq_data/{s}/{func(i)}"for i in c] for c,s in zip(df.Paths,df.ds_type)]).tolist()

In [75]:
ns = df.groupby('seq_key')['loss'].count()
losses = df.groupby('seq_key')['loss'].mean().loc[ns > 3]

losses

seq_key
07sOb7T1hUxqVoSCtqbX5g    1.170758
0k_HaRGQSjfnVlCW_yYMqg    4.015885
20GnnQaw7R2pR0a_ecPDnA    1.072728
216rnoeymisb3onr095pmn    1.217315
2gqajn4lerxvknrsnx1fbm    6.192456
                            ...   
wne465ma9irobxhskc9hcv    2.568761
x9ja8kHoqBejxFRFsqtl6Q    0.746313
yaKPBJ9QvW34iNOT453O7Q    1.639655
zt6fbpghjd3b3r53un91ke    1.229541
zuetq3q1nsyolpsid5rbfk    2.297334
Name: loss, Length: 89, dtype: float32

In [76]:
commands = []
for seq in losses.index:

    a = df.query(f'seq_key == "{seq}"')
    name = f"tmp/seqs/{seq}Ä{a.view_direction.iloc[0]}Ä{losses.loc[seq]:.2f}"
    os.makedirs(name,exist_ok=True)
    imgs = "../seq_data/train/" + np.unique(np.concatenate(a['keys'].values))  + ".jpg"

    paths = "cp " + imgs +  " " + name
    commands.extend(paths.tolist())

text = "\n".join(commands)

In [77]:
with open('copys.sh',"w") as f:
    f.write(text)

In [85]:
len(os.listdir('tmp/seqs/side')) + len(os.listdir('tmp/seqs/front'))


44

In [52]:
for seq in ns.loc[ns < 4].index:

    a = df.query(f'seq_key == "{seq}"')
    for index, row in a.iterrows():
        plot_row(row,f"tmp/saves/{index}.jpg")

In [50]:
def plot_row(row,save = None):
    fig, ax = plt.subplots(2,2,figsize=(8,8))
    keys = row['local_path']
    np.random.shuffle(keys)
    for i,a in enumerate(ax.flatten()):
        try:
            a.imshow(plt.imread(keys[i]))
        except:
            pass
        a.axis('off')
    plt.suptitle(f"{row['view_direction']} {row['loss'] :.2f}")

    if save is None:
        plt.show()
    else:
        plt.savefig(save)
        plt.close()

In [86]:
side = set([i[:i.find('Ä')]  for i in os.listdir('tmp/seqs/side/')])
front = set([i[:i.find('Ä')]  for i in os.listdir('tmp/seqs/front/')])
side = df.seq_key.apply(lambda x : x in side)
front = df.seq_key.apply(lambda x : x in front)


In [87]:
(df.loc[front,"view_direction"] == 'Sideways').sum()

120

In [88]:
(df.loc[side,"view_direction"] == 'Forward').sum()

0

In [89]:
df.loc[side,"view_direction"] = 'Sideways'
df.loc[front,"view_direction"] = 'Forward'

In [68]:
len(df)

307

In [90]:
real = pd.read_feather("new_train_seqs.df").set_index('index')
real.loc[df.index] = df
real.reset_index().to_feather("new_train_seqs.df")